In [ ]:
import os, re
import pandas as pd
import numpy as np
from lxml import etree
from tqdm.notebook import tqdm


from my_lda import LDA
import corpus_man as cm

# Downloading tycho XML

In [ ]:
!wget --no-check-certificate http://www.tycho.iel.unicamp.br/corpus/texts/xml.zip -P ./corpora
!mkdir ./corpora/xml
!unzip ./corpora/xml.zip -d ./corpora/xml

# Loading CSV Classification

In [ ]:
df = pd.read_csv('periodos_tycho.csv',header=None)
df.columns = ['period', 'file_name']
df['genre'] = None
df['predicted_period'] = None


df.period.value_counts()

In [ ]:
df.head()

In [4]:
new_df = df.sort_values(by='file_name').reset_index().drop('index',axis=1)

In [5]:
new_df.tail(10)

,period,file_name,genre,predicted_period
66,mod,va_001.txt,None,None
67,cont2,va_002.txt,None,None
68,cont2,va_003.txt,None,None
69,cont2,va_004.txt,None,None
70,cont2,va_007.txt,None,None
71,cont2,va_008.txt,None,None
72,cont1,va_009.txt,None,None
73,cont2,va_010.txt,None,None
74,cont2,va_011.txt,None,None
75,cont2,va_012.txt,None,None


In [6]:
forest=[]
for root, dirs, files in os.walk("./corpora/xml", topdown=False):
    for file_name in sorted(files):
        with open(os.path.join(root,file_name),'r') as xml:
            xml_plain = '\n'.join(xml.read().split('\n')[1:])
            try:
                forest.append(etree.fromstring(xml_plain))
            except etree.XMLSyntaxError as error:
                print(error)
                print(xml_plain)
            except ValueError:
                print(xml_plain)

In [8]:
with open('./corpora/tycho_to_lda', 'w') as dump:
    for ix, xml in enumerate(forest):
        # get id
        for i in xml:
            if i.tag == 'head':
                try:
                    name = i.get('id') + '.txt'
                except TypeError:
                    print(xml.xpath('//text()'))
                    print('%-----------------------%')
                    sys.exit()
    
        # get genre
        genre = cm.get_meta('Genre',xml).xpath('string()')
        new_df['genre'][ix] = genre
    
            
        # get text and 
        text = ' '.join(xml.xpath("//body//text()"))
    
        # process text
        text = cm.pre_process(text)
        dump.write(text)
        dump.write('\n')
        
        #print(cm.pre_process(' '.join(xml.xpath("//body//text()"))))
    

['\n   ', '\n      ', '\n         ', '\n            ', 'Author Name', '\n            ', 'Various', '\n         ', '\n         ', '\n            ', 'Author Year of Birth', '\n            ', '~1800', '\n         ', '\n         ', '\n            ', 'Original Text Title', '\n            ', '\n         ', '\n         ', '\n            ', 'Original Text Date', '\n            ', '1832-1841', '\n         ', '\n         ', '\n            ', 'Place of Birth', '\n            ', 'Africa', '\n         ', '\n         ', '\n            ', 'Original Text Place of Production', '\n            ', 'Brazil', '\n         ', '\n         ', '\n            ', 'Original Text Place of Publication', '\n            ', '\n         ', '\n         ', '\n            ', 'Genre', '\n            ', 'ata', '\n         ', '\n      ', '\n      ', '\n         ', '\n            ', 'Immediate Source Type', '\n            ', 'Transcript of original manuscript', '\n         ', '\n         ', '\n            ', 'Immediate Source E

NameError: name 'sys' is not defined

In [10]:
new_df['genre'].value_counts()

Theatre                                                                                              26
Narrative                                                                                            17
Letters                                                                                              14
Dissertation                                                                                          7
News                                                                                                  5
Narrative; Chronicle                                                                                  3
Grammar                                                                                               3
ata                                                                                                   1
Transcript of original manuscript                                                                     1
Chronica do muito alto e muito esclarecido principe D. Affonso H